In [1]:
import tensorflow as tf
import numpy as np

# $\mathrm{TensorFlow}$

# Run in Session

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y+y+2

In [3]:
sess = tf.Session() # 打开一个TensorFlow会话

In [4]:
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result, sess is tf.get_default_session())

42 False


In [5]:
sess.close() # 关闭sess会话

In [6]:
with tf.Session() as sess: # with的Block块内打开默认会话
    x.initializer.run() # 等价于 tf.get_default_session().run(x.initializer)
    y.initializer.run()
    print(f.eval(), sess.run(f), tf.get_default_session().run(f))
    print(sess is tf.get_default_session()) # True 

42 42 42
True


In [7]:
# 全局变量初始化器
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    print(f.eval())

42


In [8]:
sess = tf.InteractiveSession() # 在jupyter或者Python Shell的环境下可以使用交互式会话此时默认还是default会话
init.run()
result = f.eval()
print(result, sess is tf.get_default_session())
sess.close() # 交互式会话只能active一个

42 True


# Graphs

$\mathrm{TensorFlow}$利用图的思想，将所有操作挂在节点上, 默认操作均挂在默认图上

In [9]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph() # 默认图

True

## Manging Graphs

In [10]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2 is tf.get_default_graph()) # False 不是默认图，x2挂在graph图上
#注意在jupyter环境下所有其他的图上的节点都会在默认图上产生拷贝，此时需要经常reset内核或者
tf.reset_default_graph()

False


### Lifecycle of a Node Value
$\mathrm{TensorFlow}$的$node\ value$的生命周期是在一次**run**的期间   
在单个处理中的$\mathrm{TensorFlow}$不同会话不会共享状态，只有在分布式上状态才会存在服务器上达成共享

In [11]:
w = tf.constant(5)
x = w + 2 # 7
y = x + 5 # 12
z = x * 3 # 21

In [12]:
with tf.Session() as sess:
    print(y.eval(), z.eval()) # 实际上TensorFlow在计算y后，会丢弃掉图上的节点值，重新计算z依赖的变量
    print(sess.run(y), sess.run(z))
    print(sess.run([y, z]))   # 这样在一次run中才能重复利用节点值

12 21
12 21
[12, 21]


In [33]:
a = np.array([[1],[2],[3]])
b = np.array([[4],[5],[6]])
print(a.shape, b.shape)
np.c_[a, b], np.c_[a, b].shape

(3, 1) (3, 1)


(array([[1, 4],
        [2, 5],
        [3, 6]]), (3, 2))

# Linear Regression with TensorFlow

In [14]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
print("scale:", m, n)
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')

XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # 闭解

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value, '\ntheta\'s scale:', theta_value.shape)

scale: 20640 8
[[-3.7383255e+01]
 [ 4.3574786e-01]
 [ 9.3480907e-03]
 [-1.0648697e-01]
 [ 6.4317447e-01]
 [-4.2286665e-06]
 [-3.7746700e-03]
 [-4.2583770e-01]
 [-4.3952349e-01]] 
theta's scale: (9, 1)


In [15]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
data = housing.data

housing_data_plus_bias = np.c_[np.ones((m,1)), data]
X_train, X_val, y_train, y_val = train_test_split(housing_data_plus_bias, housing.target.reshape(-1,1))

X_train = tf.constant(X_train, dtype=tf.float32, name='X_train')
X_val = tf.constant(X_val, dtype=tf.float32, name='X_val')

y_train = tf.constant(y_train, dtype=tf.float32, name='y_train')
y_val = tf.constant(y_val, dtype=tf.float32, name='y_val')

X_train_T = tf.transpose(X_train)

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(X_train_T, X_train)), X_train_T), y_train)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)
    y_pred = tf.matmul(X_val, theta_value)
    error = y_pred - y_val
    mse = tf.reduce_mean(tf.square(error), name="mse")
    print('预测:\n', y_pred.eval(),'\n真实标记:\n', y_val.eval(),'\n均方误差:',np.sqrt(np.sum(error.eval()**2)/(m-1)), mse.eval())

[[-3.6671883e+01]
 [ 4.3290150e-01]
 [ 9.2395172e-03]
 [-1.0443940e-01]
 [ 6.4053464e-01]
 [-7.9397687e-06]
 [-3.9392635e-03]
 [-4.1903567e-01]
 [-4.3165210e-01]]
预测:
 [[2.461358  ]
 [0.34167862]
 [2.1159124 ]
 ...
 [1.3530731 ]
 [0.7720909 ]
 [2.8207245 ]] 
真实标记:
 [[4.889]
 [1.078]
 [1.503]
 ...
 [1.276]
 [1.01 ]
 [4.857]] 
均方误差: 0.36666975971019805 0.5377607


### Implementing GD

In [16]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
data = housing.data
m, n = data.shape
print('data shape: ', m, '*', n)
target = housing.target
scaled_data =  StandardScaler().fit_transform(data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_data]
print(np.mean(scaled_housing_data_plus_bias))

X_train, X_val, y_train, y_val = train_test_split(scaled_housing_data_plus_bias, target.reshape(-1,1))
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

# 注意Tensor不是占位符的合法替代，直接使用np.array即可
# X_train = tf.constant(X_train, dtype=tf.float32, name='X_train')
# X_val   = tf.constant(X_val,   dtype=tf.float32, name='X_val')
# y_train = tf.constant(y_train, dtype=tf.float32, name='y_train')
# y_val   = tf.constant(y_val,   dtype=tf.float32, name='y_val')

X = tf.placeholder(tf.float32,shape=(None, 9),  name="X")
Y = tf.placeholder(tf.float32,shape=(None, 1), name="Y")
Y_Val = tf.placeholder(tf.float32,shape=(None, 1), name="Y_val")

data shape:  20640 * 8
0.11111111111111005
(15480, 9) (15480, 1) (5160, 9) (5160, 1)


In [17]:
%%time

n_epochs = 1200
lr = 0.01
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error =  y_pred - Y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_ops = tf.assign(theta, theta - lr * gradients) # geadient descent: theta = theta - lr * gradients
init = tf.global_variables_initializer()
print(theta)

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", str(epoch).zfill(4), '<==============> MES =', mse.eval(feed_dict={X:X_train, Y:y_train}))
        training_ops.eval(feed_dict={X:X_train, Y:y_train})
    best_theta = theta.eval()

    print('val_mse =', mse.eval(feed_dict={X:X_val, Y:y_val}))
print('best_theta:', best_theta.flatten())

<tf.Variable 'theta:0' shape=(9, 1) dtype=float32_ref>
Epoch 0000 <==============> MES = 3.9040933
Epoch 0100 <==============> MES = 0.6593812
Epoch 0200 <==============> MES = 0.5448185
Epoch 0300 <==============> MES = 0.5353779
Epoch 0400 <==============> MES = 0.5323018
Epoch 0500 <==============> MES = 0.53024447
Epoch 0600 <==============> MES = 0.52868503
Epoch 0700 <==============> MES = 0.5274715
Epoch 0800 <==============> MES = 0.5265194
Epoch 0900 <==============> MES = 0.5257696
Epoch 1000 <==============> MES = 0.52517927
Epoch 1100 <==============> MES = 0.5247141
val_mse = 0.5331955
best_theta: [ 2.0635517e+00  8.2997817e-01  1.2403106e-01 -2.5593805e-01
  2.6323408e-01 -1.6935226e-03 -4.8034508e-02 -7.9701191e-01
 -7.6936400e-01]
CPU times: user 2.29 s, sys: 169 ms, total: 2.46 s
Wall time: 1.51 s


### Autodiff
$\mathrm{TensorFlow}$使用$\mathrm{Reverse-mode\ autodiff}$自动化求梯度, 原理见附录D

In [18]:
%%time

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
data = housing.data
m, n = data.shape
print(m, n)
target = housing.target
scaled_data =  StandardScaler().fit_transform(data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_data]
print(np.mean(scaled_housing_data_plus_bias))

X_train, X_val, y_train, y_val = train_test_split(scaled_housing_data_plus_bias, target.reshape(-1,1))
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

n_epochs = 1200
lr = 0.01

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name='mse')
# gradients = tf.gradients(mse, [theta])[0] # 自动求偏导，输入函数，【待求偏导的变量列表】-》列表
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr) # TensorFlow 自带的梯度下降优化器
# optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9) # 动量优化梯度下降
training_ops = optimizer.minimize(mse)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch-{} <==============> MES = {}".format(str(epoch).zfill(4) ,mse.eval(feed_dict={X:X_train, y:y_train})))
        sess.run(training_ops, feed_dict={X:X_train, y:y_train})
    best_theta = theta.eval()
    print('val_mse =', mse.eval(feed_dict={X:X_val, y:y_val}))
    print('best_theta:', best_theta.flatten())

20640 8
0.11111111111111005
(15480, 9) (15480, 1) (5160, 9) (5160, 1)
Epoch-0000 <==============> MES = 13.142555236816406
Epoch-0100 <==============> MES = 0.8662416338920593
Epoch-0200 <==============> MES = 0.6112032532691956
Epoch-0300 <==============> MES = 0.5796080827713013
Epoch-0400 <==============> MES = 0.5628477931022644
Epoch-0500 <==============> MES = 0.5513385534286499
Epoch-0600 <==============> MES = 0.5431396961212158
Epoch-0700 <==============> MES = 0.5372486114501953
Epoch-0800 <==============> MES = 0.5330027937889099
Epoch-0900 <==============> MES = 0.5299386978149414
Epoch-1000 <==============> MES = 0.5277246236801147
Epoch-1100 <==============> MES = 0.5261224508285522
val_mse = 0.5367589
best_theta: [ 2.0632253   0.8093559   0.13925464 -0.18233489  0.21208344  0.00451868
 -0.05332402 -0.7908571  -0.7553287 ]
CPU times: user 3.97 s, sys: 285 ms, total: 4.26 s
Wall time: 2.49 s


# Implementing Mini-batch GD
使用占位符$(placeholder)$ 来$pass$训练数据模型

In [19]:
A = tf.placeholder(tf.float32, shape=(None, 3)) # 第一维度任意填充
B = A + 5
with tf.Session() as sess:
    A_val = A.eval(feed_dict={A:[[1,2,3], [2,3,4]]}) # can feed any ops not only placeholds
    B_val_1 = B.eval(feed_dict={A:A_val})
    B_val_2 = B.eval(feed_dict={A:[[1,2,3], [2,3,4], [3,4,5]]})
print(A_val, B_val_1, B_val_2, sep='\n\n')

[[1. 2. 3.]
 [2. 3. 4.]]

[[6. 7. 8.]
 [7. 8. 9.]]

[[ 6.  7.  8.]
 [ 7.  8.  9.]
 [ 8.  9. 10.]]


to implementing mini-batch GD modify X_train, y_train by placeholds

In [20]:
%%time 

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
data = housing.data
target = housing.target
m, n = data.shape

scaled_data = StandardScaler().fit_transform(data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_data]

X_train_tot, X_val, y_train_tot, y_val = train_test_split(scaled_housing_data_plus_bias, target.reshape(-1, 1))

# X_val = tf.constant(X_val, dtype=tf.float32, name='X_val')
# y_val = tf.constant(y_val, dtype=tf.float32, name='y_val')

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')  # placeholders
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

n_epochs = 100
lr = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):  # 随机生成batch
    np.random.seed(epoch * n_batches + batch_index)  # 生成不同的种子
    indices = np.random.randint(15480, size=batch_size)
    X_batch = X_train_tot[indices]
    y_batch = y_train_tot.reshape(-1,1)[indices]
    return X_batch, y_batch


theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
# gradients = tf.gradients(mse, [theta])[0]  # autodiff auto cal gradients
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)  # Use TensorFlow GD optimizer
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver() # 保存节点

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if epoch % 100 == 0:
                save_path = saver.save(sess, './tmp/my_model.ckpt')
    best_theta = theta.eval()
    
    y_val_pred = y_pred.eval(feed_dict={X:X_val})
    val_mse = mse.eval(feed_dict={y_pred:y_val_pred, y:y_val})
#     y_val_pred = tf.matmul(X_val, best_theta, name='y_val_pred')
#     mse = tf.reduce_mean(tf.square(y_val_pred - y_val), name='mse')
    print('val_mse =', val_mse)
    print('best_theta:', best_theta.flatten())
    save_path = saver.save(sess, './tmp/my_final_model.ckpt')
    # saver.restore(sess, './tmp/my_final_model.ckpt')
    # saver = tf.train.Saver({'weigh': theta}) # 以字典形式保存特定节点变量

val_mse = 0.5199688
best_theta: [ 2.0752447   0.8302172   0.11966775 -0.32165644  0.19795075 -0.00899185
 -0.02732604 -0.91518587 -0.88679576]
CPU times: user 35.8 s, sys: 4.63 s, total: 40.4 s
Wall time: 33.8 s


In [21]:
with tf.Session() as sess:
    saver.restore(sess, './tmp/my_final_model.ckpt')
    restored_best_theta = theta.eval()
    print(np.allclose(best_theta, restored_best_theta))

INFO:tensorflow:Restoring parameters from ./tmp/my_final_model.ckpt
True


In [34]:
%%time
import numpy as np
import tensorflow as tf
from datetime import datetime
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
data = housing.data
target = housing.target
m, n = data.shape

scaled_data = StandardScaler().fit_transform(data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_data]

X_train_tot, X_val, y_train_tot, y_val = train_test_split(scaled_housing_data_plus_bias, target.reshape(-1, 1))

now = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
print('Run in', now)
root_logdir = './tmp/tf_logs'
logdir = "{}/run-{}/".format(root_logdir, now)
tf.reset_default_graph() # 生成日志时一定要先清除默认图

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

lr = 0.01
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

# gradients = 2 / m * tf.matmul(tf.transpose(X), error)
# training_op = tf.assign(theta, theta - lr * gradients) # geadient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar(name='MSE', tensor=mse) # 创建标量的日志节点
file_writer = tf.summary.FileWriter(logdir=logdir, graph=tf.get_default_graph()) # 创建写文件夹的节点

n_epochs = 100
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):  # 随机生成batch
    np.random.seed(epoch * n_batches + batch_index)  # 生成不同的种子
    indices = np.random.randint(15480, size=batch_size)
    X_batch = X_train_tot[indices]
    y_batch = y_train_tot.reshape(-1,1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)                                       
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step) # 追写日志+训练步数
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    y_val_pred = y_pred.eval(feed_dict={X:X_val})
    val_mse = mse.eval(feed_dict={y_pred:y_val_pred, y:y_val})
    print("val mse:", val_mse)
    file_writer.close()
    print('Finish! 👌')

Run in 2019_03_01_11_37_30
val mse: 0.4936723
Finish! 👌
CPU times: user 24.7 s, sys: 2.27 s, total: 27 s
Wall time: 18 s


### Modularity

使用TensorFlow模块化操作，设计五个整流线性单元$rectified\ linear\ units(ReLU)$

In [23]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, 0.0, name='relu')

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name='output')

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(output.eval(feed_dict={X: np.array([[1,2,3]])}))

[[8.860965]]


### Sharing Variables
1. 使用变量空间，创建特定空间的变量

In [24]:
import tensorflow as tf
tf.reset_default_graph()

with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshold', shape=(), initializer=tf.constant_initializer(0.0))
    print(threshold.op.name)

relu/threshold


In [25]:
with tf.variable_scope('relu', reuse=True): # 使用tf.variable_scope reuse=True 创建可重用变量空间
    threshold = tf.get_variable("threshold")
    # 默认一个变量不可reuse除非指其 set reuse=True or reuse=tf.AUTO_REUSE in VarScope
print(threshold.op.name)

relu/threshold


In [27]:
with tf.variable_scope('relu') as scope: # 或者用reuse_variables属性打开可重用
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")
    print(threshold.op.name)

relu/threshold


将relu变量空间中threshold传给多个ReLU

In [28]:
import tensorflow as tf
tf.reset_default_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    with tf.variable_scope('relu', reuse=True):
        threshod = tf.get_variable("threshold") # 调用relu区域的 共享变量
    return tf.maximum(z, threshold, name='max')

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

with tf.variable_scope('relu'):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
    
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name='output')

这样的属性看起来不属于ReLU, 所以可以在一个整流器上建立变量，在其他整流器上共享

In [29]:
tf.reset_default_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    threshold = tf.get_variable('threshold', shape=(), initializer=tf.constant_initializer(0.0)) # 初次创建，其他reuse不必再次申明shape等
    return tf.maximum(z, threshold, name='max')

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = []
for relu_idx in range(5):
    with tf.variable_scope('relu', reuse=(relu_idx >= 1)) as scope: # 第一次创建变量，其余整流器reuse=True
        relus.append(relu(X))
output = tf.add_n(relus, name='output')

In [30]:
a_val = tf.Variable(5)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(a_val.eval(), sess.run(a_val))

5 5


In [31]:
import tensorflow as tf

x = tf.Variable(tf.random_uniform(shape=(), maxval=1.0, minval=0.0))
x_new_val = tf.placeholder(shape=(), dtype=tf.float32)
x_assign = tf.assign(x, x_new_val)

with tf.Session() as sess:
    sess.run(x.initializer)
    print(x.eval())
#     x_assign.eval(feed_dict={x_new_val:5.5})
    sess.run(x_assign, feed_dict={x_new_val:5.5})
    print(x.eval())

0.3446319
5.5


In [32]:
import pip
from pip._internal.utils.misc import get_installed_distributions

for dist in get_installed_distributions():
    print(dist.project_name)

kaggle
zope.interface
wxPython
wheel
Werkzeug
webencodings
wcwidth
Twisted
traitlets
tornado
torchvision
torch
toolz
testpath
terminado
termcolor
tensorflow
tensorflow-probability
tensorboard
six
sip
simplegeneric
setuptools
service-identity
Send2Trash
scipy
scikit-learn
scikit-image
pyzmq
PyYAML
PyWavelets
pytz
python-dateutil
pyparsing
pyOpenSSL
Pygments
pycparser
pycodestyle
pyasn1
pyasn1-modules
ptyprocess
psutil
protobuf
prompt-toolkit
prometheus-client
pip
Pillow
pickleshare
pexpect
parso
pandocfilters
pandas
olefile
numpy
notebook
nose
networkx
nbformat
nbconvert
mkl-random
mkl-fft
mistune
metakernel
memory-profiler
matplotlib
matlabengineforpython
matlab-kernel
MarkupSafe
Markdown
line-profiler
kiwisolver
Keras
Keras-Preprocessing
Keras-Applications
jupyterlab
jupyterlab-server
jupyterlab-launcher
jupyter-core
jupyter-client
jsonschema
Jinja2
jedi
ipython
ipython-genutils
ipykernel
incremental
imageio
idna
hyperlink
html5lib
h5py
grpcio
gast
entrypoints
decorator
dask
cytoolz
c